In [1]:
from IPython.display import clear_output
clear_output(wait=True)

import sys, socket
from pathlib import Path

ROOT = Path.cwd().resolve().parent             
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from dotenv import load_dotenv
load_dotenv(dotenv_path=ROOT / ".env")

# --- Imports ---
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd

try:
    JupyterDash._server_threads.clear()
except Exception:
    pass

from animal_shelter import AnimalShelter  

# Connect 
db = AnimalShelter()                                   # uses .env (local Mongo)
df_all = pd.DataFrame.from_records(db.read({}))        # dataset 

if "Unnamed: 0" in df_all.columns:
    df_all = df_all.drop(columns=["Unnamed: 0"])

# Logo 
logo_path = ROOT / "assets" / "Grazioso_Salvare_Logo.png"
encoded_logo = base64.b64encode(logo_path.read_bytes()).decode()

# Filters
WATER_RESCUE = {
    "$and": [
        {"animal_type": "Dog"},
        {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
        {"sex_upon_outcome": "Intact Female"},
        {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
    ]
}
MOUNTAIN_WILDERNESS = {
    "$and": [
        {"animal_type": "Dog"},
        {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
        {"sex_upon_outcome": "Intact Male"},
        {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
    ]
}
DISASTER_TRACKING = {
    "$and": [
        {"animal_type": "Dog"},
        {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
        {"sex_upon_outcome": "Intact Male"},
        {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}},
    ]
}

# layout 
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Img(src=f"data:image/png;base64,{encoded_logo}", style={"height": "100px"}),
    html.Center(html.B(html.H1("SNHU CS-340 Dashboard — Jesse Kuczynski"))),
    html.Hr(),
    dcc.RadioItems(
        id="filter-type",
        options=[
            {"label": "Water Rescue", "value": "water"},
            {"label": "Mountain/Wilderness Rescue", "value": "mount"},
            {"label": "Disaster Rescue & Tracking", "value": "disaster"},
            {"label": "Reset", "value": "reset"},
        ],
        value="reset",
        inline=True,
    ),
    html.Hr(),
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df_all.columns],
        data=df_all.to_dict("records"),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        page_action="native",
        page_current=0,
        page_size=15,  
        selected_rows=[0],
        style_cell={"whiteSpace": "normal", "height": "auto"},  # wrap long text
    ),
    html.Br(),
    html.Hr(),
    html.Div(style={"display": "flex", "gap": "16px"}, children=[
        html.Div(id="graph-id", style={"flex": "1"}),
        html.Div(id="map-id",   style={"flex": "1"}),
    ]),
])

# Callbacks
@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "columns"),
    Input("filter-type", "value"),
)
def update_table(filter_value: str):
    if filter_value == "water":
        query = WATER_RESCUE
    elif filter_value == "mount":
        query = MOUNTAIN_WILDERNESS
    elif filter_value == "disaster":
        query = DISASTER_TRACKING
    else:
        query = {}

    dff = pd.DataFrame.from_records(db.read(query))
    if "Unnamed: 0" in dff.columns:
        dff = dff.drop(columns=["Unnamed: 0"])
    cols = [{"name": c, "id": c, "deletable": False, "selectable": True} for c in dff.columns]
    return dff.to_dict("records"), cols

@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_viewport_data"),
)
def update_graph(view_rows):
    dff = pd.DataFrame(view_rows or [])
    if dff.empty or "breed" not in dff.columns:
        return [html.Div("No data to plot")]
    counts = dff["breed"].value_counts().nlargest(20).reset_index()
    counts.columns = ["breed", "count"]
    fig = px.bar(counts, x="breed", y="count", title="Top 20 Breeds (visible rows)")
    fig.update_xaxes(tickangle=45)
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "derived_viewport_data"),
    Input("datatable-id", "derived_viewport_selected_rows"),
)
def update_map(view_rows, sel_rows):
    dff = pd.DataFrame(view_rows or [])
    if dff.empty:
        return [html.Div("No data")]
    idx = (sel_rows or [0])[-1]
    idx = max(0, min(idx, len(dff) - 1))

    lat_col = next((c for c in dff.columns if c.lower() in ("location_lat", "latitude", "lat")), None)
    lon_col = next((c for c in dff.columns if c.lower() in ("location_long", "longitude", "lon", "lng")), None)
    if lat_col and lon_col:
        lat, lon = dff.loc[idx, lat_col], dff.loc[idx, lon_col]
    else:
        lat, lon = dff.iloc[idx, 13], dff.iloc[idx, 14]  # fallback indices

    name = str(dff.iloc[idx].get("name", ""))
    breed = str(dff.iloc[idx].get("breed", ""))

    return [dl.Map(
        style={"width": "100%", "height": "500px"},
        center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H4("Animal"), html.P(name)])
            ])
        ])
    ]

# Run inline 
s = socket.socket(); s.bind(('', 0)); PORT = s.getsockname()[1]; s.close()
app.run_server(mode="inline", port=PORT, debug=False)


C:\Users\Kuczy\PycharmProjects\Databases\venv\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

